In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#!pip install bs4

In [3]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

In [5]:
tables = soup.find_all('table') # in html table is represented by the tag <table>

In [6]:
dataframe_list = pd.read_html(url, flavor='bs4')

In [7]:
neighborhood=dataframe_list[10]

In [8]:
neighborhood.drop(['CDN number','Neighbourhoods covered','Map','Unnamed: 5'],axis = 1 ,inplace=True)

In [9]:
neighborhood.rename(columns={'City-designated neighbourhood':'Neighborhood','Former city/borough':'Borough'},inplace=True)

In [10]:
neighborhood['Neighborhood'] = neighborhood['Neighborhood']+',Toronto'
neighborhood_list = neighborhood['Neighborhood'].tolist()

In [11]:
geolocator = Nominatim(user_agent="mygeocoder")
coordinates_of_neighborhood = []
for i in neighborhood_list:
    location = geolocator.geocode(i.format(i))
    if location:
        coordinates_of_neighborhood.append([i, location.latitude, location.longitude])

In [12]:
data_frame_of_coordinates = pd.DataFrame(coordinates_of_neighborhood, columns=["Neighborhood", "Latitude","Longitude"])

In [13]:
data_frame_of_coordinates['Borough']=neighborhood['Borough']

In [16]:
data_frame_of_coordinates.head()
neighborhoods = data_frame_of_coordinates

In [15]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
neighborhoods['Borough'].value_counts()

Old City of Toronto    33
North York             26
Scarborough            20
Etobicoke              16
York                    9
East York               6
Name: Borough, dtype: int64

In [19]:
CLIENT_ID = 'ELMEJVES0D2UHDEQXRIK00A2UO3OPPGSSVMDRWZELQAABP3O' # your Foursquare ID
CLIENT_SECRET = '3F34BDQH2USRLMZNP4QF5KFXTGMD4W5QMXWCAWITWX3LQONB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ELMEJVES0D2UHDEQXRIK00A2UO3OPPGSSVMDRWZELQAABP3O
CLIENT_SECRET:3F34BDQH2USRLMZNP4QF5KFXTGMD4W5QMXWCAWITWX3LQONB


In [20]:
neighborhoods.loc[0, 'Neighborhood']

'Agincourt North,Toronto'

In [21]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Agincourt North,Toronto are 43.8080383, -79.2664391.


In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ELMEJVES0D2UHDEQXRIK00A2UO3OPPGSSVMDRWZELQAABP3O&client_secret=3F34BDQH2USRLMZNP4QF5KFXTGMD4W5QMXWCAWITWX3LQONB&v=20180605&ll=43.8080383,-79.2664391&radius=500&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60eed92967adc21850f94383'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Agincourt',
  'headerFullLocation': 'Agincourt, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 43.8125383045, 'lng': -79.26021513985647},
   'sw': {'lat': 43.8035382955, 'lng': -79.27266306014353}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51fe9e21498ecbfb4fef45c1',
       'name': "Menchie's",
       'location': {'address': '1571 Sandhurst Circle',
        'lat': 43.80833791536094,
        'lng': -79.26828825119229,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80833791536094

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-25-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Menchie's,Frozen Yogurt Shop,43.808338,-79.268288
1,Saravanaa Bhavan South Indian Restaurant,Indian Restaurant,43.810117,-79.269275
2,Booster Juice,Juice Bar,43.809915,-79.269382
3,Shoppers Drug Mart,Pharmacy,43.808894,-79.269854
4,Dollarama,Discount Store,43.808894,-79.269854


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Agincourt North,Toronto
Agincourt South-Malvern West,Toronto
Alderwood,Toronto
Annex,Toronto
Banbury-Don Mills,Toronto
Bathurst Manor,Toronto
Bay Street Corridor,Toronto
Bayview Village,Toronto
Bayview Woods-Steeles,Toronto
Bendale,Toronto
Black Creek,Toronto
Broadview North,Toronto
Cabbagetown-South St. James Town,Toronto
Casa Loma,Toronto
Centennial Scarborough,Toronto
Church-Yonge Corridor,Toronto
Clanton Park,Toronto
Cliffcrest,Toronto
Corso Italia-Davenport,Toronto
Crescent Town,Toronto
Danforth - East York,Toronto
Danforth Village - Toronto,Toronto
Don Valley Village,Toronto
Dorset Park,Toronto
Dufferin Grove,Toronto
East End-Danforth,Toronto
Edenbridge-Humber Valley,Toronto
Eglinton East,Toronto
Elms-Old Rexdale,Toronto
Englemount-Lawrence,Toronto
Etobicoke West Mall,Toronto
Flemingdon Park,Toronto
Forest Hill North,Toronto
Forest Hill South,Toronto
Glenfield-Jane Heights,Toronto
Guildwood,Toronto
Henry Farm,Toronto
High Park North,Toronto
High Park-Swansea,Toronto
Highland Cree

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(2313, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Agincourt North,Toronto",43.808038,-79.266439,Menchie's,43.808338,-79.268288,Frozen Yogurt Shop
1,"Agincourt North,Toronto",43.808038,-79.266439,Saravanaa Bhavan South Indian Restaurant,43.810117,-79.269275,Indian Restaurant
2,"Agincourt North,Toronto",43.808038,-79.266439,Booster Juice,43.809915,-79.269382,Juice Bar
3,"Agincourt North,Toronto",43.808038,-79.266439,Shoppers Drug Mart,43.808894,-79.269854,Pharmacy
4,"Agincourt North,Toronto",43.808038,-79.266439,Dollarama,43.808894,-79.269854,Discount Store


In [31]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Agincourt North,Toronto",24,24,24,24,24,24
"Agincourt South-Malvern West,Toronto",14,14,14,14,14,14
"Alderwood,Toronto",8,8,8,8,8,8
"Annex,Toronto",40,40,40,40,40,40
"Banbury-Don Mills,Toronto",4,4,4,4,4,4
"Bathurst Manor,Toronto",4,4,4,4,4,4
"Bay Street Corridor,Toronto",100,100,100,100,100,100
"Bayview Village,Toronto",12,12,12,12,12,12
"Bayview Woods-Steeles,Toronto",1,1,1,1,1,1


In [33]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


In [45]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Transportation Service,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [36]:
toronto_onehot.shape

(2313, 268)

In [46]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Transportation Service,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Agincourt North,Toronto",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0

In [47]:
toronto_grouped.shape

(110, 268)

In [48]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North,Toronto----
                venue  freq
0                Bank  0.08
1  Chinese Restaurant  0.08
2       Movie Theater  0.04
3        Liquor Store  0.04
4          Beer Store  0.04


----Agincourt South-Malvern West,Toronto----
                    venue  freq
0              Restaurant  0.21
1          Breakfast Spot  0.07
2        Department Store  0.07
3               Bookstore  0.07
4  Furniture / Home Store  0.07


----Alderwood,Toronto----
         venue  freq
0  Pizza Place  0.25
1     Pharmacy  0.12
2          Gym  0.12
3          Pub  0.12
4  Coffee Shop  0.12


----Annex,Toronto----
              venue  freq
0       Pizza Place  0.10
1   Thai Restaurant  0.05
2               Gym  0.05
3            Bistro  0.05
4  Sushi Restaurant  0.05


----Banbury-Don Mills,Toronto----
                 venue  freq
0          Gas Station  0.25
1               Resort  0.25
2                 Park  0.25
3  Japanese Restaurant  0.25
4                  ATM  0.00


----Bathurst Ma

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt North,Toronto",Bank,Chinese Restaurant,Fast Food Restaurant,Frozen Yogurt Shop,Juice Bar,Fried Chicken Joint,Beer Store,Sporting Goods Shop,Spa,Coffee Shop
1,"Agincourt South-Malvern West,Toronto",Restaurant,Intersection,Bookstore,Department Store,Japanese Restaurant,Electronics Store,Salad Place,Furniture / Home Store,Gym / Fitness Center,Sporting Goods Shop
2,"Alderwood,Toronto",Pizza Place,Pharmacy,Coffee Shop,Dance Studio,Gym,Sandwich Place,Pub,Donut Shop,Discount Store,Distribution Center
3,"Annex,Toronto",Pizza Place,Park,Thai Restaurant,Bistro,Sushi Restaurant,Gym,Pharmacy,Coffee Shop,Pastry Shop,Korean Restaurant
4,"Banbury-Don Mills,Toronto",Park,Resort,Japanese Restaurant,Gas Station,Eastern European Restaurant,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant


In [52]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 3, 0, 0, 2, 0], dtype=int32)

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt North,Toronto",43.808038,-79.266439,Scarborough,0,Bank,Chinese Restaurant,Fast Food Restaurant,Frozen Yogurt Shop,Juice Bar,Fried Chicken Joint,Beer Store,Sporting Goods Shop,Spa,Coffee Shop
1,"Agincourt South-Malvern West,Toronto",43.781969,-79.257689,Scarborough,0,Restaurant,Intersection,Bookstore,Department Store,Japanese Restaurant,Electronics Store,Salad Place,Furniture / Home Store,Gym / Fitness Center,Sporting Goods Shop
2,"Alderwood,Toronto",43.601717,-79.545232,Etobicoke,0,Pizza Place,Pharmacy,Coffee Shop,Dance Studio,Gym,Sandwich Place,Pub,Donut Shop,Discount Store,Distribution Center
3,"Annex,Toronto",43.670338,-79.407117,Old City of Toronto,0,Pizza Place,Park,Thai Restaurant,Bistro,Sushi Restaurant,Gym,Pharmacy,Coffee Shop,Pastry Shop,Korean Restaurant
4,"Banbury-Don Mills,Toronto",43.752683,-79.365270,North York,1,Park,Resort,Japanese Restaurant,Gas Station,Eastern European Restaurant,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant


In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.808038,Bank,Chinese Restaurant,Fast Food Restaurant,Frozen Yogurt Shop,Juice Bar,Fried Chicken Joint,Beer Store,Sporting Goods Shop,Spa,Coffee Shop
1,43.781969,Restaurant,Intersection,Bookstore,Department Store,Japanese Restaurant,Electronics Store,Salad Place,Furniture / Home Store,Gym / Fitness Center,Sporting Goods Shop
2,43.601717,Pizza Place,Pharmacy,Coffee Shop,Dance Studio,Gym,Sandwich Place,Pub,Donut Shop,Discount Store,Distribution Center
3,43.670338,Pizza Place,Park,Thai Restaurant,Bistro,Sushi Restaurant,Gym,Pharmacy,Coffee Shop,Pastry Shop,Korean Restaurant
6,43.668865,Coffee Shop,Italian Restaurant,Boutique,Japanese Restaurant,Hotel,Restaurant,French Restaurant,Sushi Restaurant,Café,Indian Restaurant
7,43.769197,Bank,Fast Food Restaurant,Fish Market,Breakfast Spot,Gas Station,Outdoor Supply Store,Sandwich Place,Metro Station,Sporting Goods Shop,Persian Restaurant
9,43.753520,Optical Shop,Intersection,Tennis Court,Dog Run,Discount Store,Chinese Restaurant,Grocery Store,Fast Food Restaurant,Cuban Restaurant,Construction & Landscaping
10,43.695400,ATM,Coffee Shop,Construction & Landscaping,Intersection,Playground,Falafel Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Discount Store
11,43.683922,Intersection,Ice Cream Shop,Theater,Other Great Outdoors,Bus Line,Grocery Store,Park,Discount Store,Coffee Shop,Electronics Store
12,43.663703,Restaurant,Coffee Shop,Café,Gastropub,Market,Pub,Pizza Place,Diner,Beer Store,Bakery


In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,43.752683,Park,Resort,Japanese Restaurant,Gas Station,Eastern European Restaurant,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant
19,43.695403,Metro Station,Park,Convenience Store,Golf Course,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
29,43.713358,Bakery,Coffee Shop,Shoe Store,Park,Shopping Mall,Egyptian Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
36,43.769509,Intersection,Women's Store,Tennis Court,Park,Yoga Studio,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
37,43.657383,Park,Mattress Store,Baseball Field,Yoga Studio,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School
38,43.638093,Park,Gym / Fitness Center,Light Rail Station,Bus Stop,Gas Station,Beach,Trail,Intersection,Lake,American Restaurant
39,43.790117,IT Services,Construction & Landscaping,Park,Yoga Studio,Electronics Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
42,43.760078,Bakery,Park,Shopping Mall,Yoga Studio,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Distribution Center
44,43.735990,Park,Intersection,Metro Station,Deli / Bodega,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Yoga Studio
59,43.631239,Park,Piano Bar,Golf Course,Yoga Studio,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant


In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,43.798127,Dog Run,Yoga Studio,Distribution Center,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Elementary School


In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,43.763893,Park,Playground,Baseball Field,Convenience Store,Yoga Studio,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
32,43.693559,Park,Playground,Bank,Yoga Studio,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
33,43.693559,Park,Playground,Bank,Yoga Studio,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
51,43.758950,Playground,Spa,Yoga Studio,Egyptian Restaurant,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Elementary School
91,43.816178,Playground,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Elementary School,Electronics Store,Egyptian Restaurant


In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,43.787491,Park,Yoga Studio,Egyptian Restaurant,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Discount Store
26,43.670672,Garden,Park,Yoga Studio,Egyptian Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Distribution Center
